# Scraping sewakost.com data

## Preparations

In [1]:
# import some libraries
import re
import codecs
import time
import pyprind
import pandas as pd
import numpy as np

import requests 
from bs4 import BeautifulSoup

## Get all kos urls

In [ ]:
# Getting kos urls
# max_page were initiated manually by checking max page for each loc

loc = 'all'

if loc == 'jakarta':
    page_url = 'https://www.sewakost.com/kost/search-results.html?action=search&post_form_key=listings_advanced&f%5BCategory_ID%5D=8&f%5Bcategory_parent_ids%5D=8&f%5Bsort_by%5D=0&f%5Bsort_type%5D=asc&search=Cari'
    max_page = 246
elif loc == 'bandung':
    page_url = 'https://www.sewakost.com/kost/search-results.html?form=listings_quick&pills_listings_quick=listings_quick&f%5BCategory_ID%5D=35&f%5Bcategory_parent_ids%5D=17%2C35'
    max_page = 52
elif loc == 'surabaya':
    page_url = 'https://www.sewakost.com/kost/search-results.html?action=search&post_form_key=listings_advanced&f%5Bkeyword_search%5D=&f%5Bkeyword_search_type%5D=2&f%5BCategory_ID%5D=37&f%5Bcategory_parent_ids%5D=19%2C37&f%5Bprice%5D%5Bfrom%5D=&f%5Bprice%5D%5Bto%5D=&f%5Bsort_by%5D=0&f%5Bsort_type%5D=asc&search=Cari'
    max_page = 59
elif loc == 'malang':
    page_url = 'https://www.sewakost.com/kost/search-results.html?action=search&post_form_key=listings_advanced&f%5Bkeyword_search%5D=&f%5Bkeyword_search_type%5D=2&f%5BCategory_ID%5D=64&f%5Bcategory_parent_ids%5D=19%2C64&f%5Bprice%5D%5Bfrom%5D=&f%5Bprice%5D%5Bto%5D=&f%5Bsort_by%5D=0&f%5Bsort_type%5D=asc&search=Cari'
    max_page = 28
elif loc == 'jogja':
    page_url = 'https://www.sewakost.com/kost/search-results.html?action=search&post_form_key=listings_advanced&f%5Bkeyword_search%5D=&f%5Bkeyword_search_type%5D=2&f%5BCategory_ID%5D=85&f%5Bcategory_parent_ids%5D=20%2C85&f%5Bprice%5D%5Bfrom%5D=&f%5Bprice%5D%5Bto%5D=&f%5Bsort_by%5D=0&f%5Bsort_type%5D=asc&search=Cari'
    max_page = 8
elif loc == 'semarang':
    page_url = 'https://www.sewakost.com/kost/search-results.html?action=search&post_form_key=listings_advanced&f%5Bkeyword_search%5D=&f%5Bkeyword_search_type%5D=2&f%5BCategory_ID%5D=36&f%5Bcategory_parent_ids%5D=18%2C36&f%5Bprice%5D%5Bfrom%5D=&f%5Bprice%5D%5Bto%5D=&f%5Bsort_by%5D=0&f%5Bsort_type%5D=asc&search=Cari'
    max_page = 35
elif loc == 'solo':
    page_url = 'https://www.sewakost.com/kost/search-results.html?action=search&post_form_key=listings_advanced&f%5Bkeyword_search%5D=&f%5Bkeyword_search_type%5D=2&f%5BCategory_ID%5D=91&f%5Bcategory_parent_ids%5D=18%2C91&f%5Bprice%5D%5Bfrom%5D=&f%5Bprice%5D%5Bto%5D=&f%5Bsort_by%5D=0&f%5Bsort_type%5D=asc&search=Cari'
    max_page = 7
elif loc == 'all':
    page_url = 'https://www.sewakost.com/kost/search-results.html?form=listings_quick&pills_listings_quick=listings_quick&f%5BCategory_ID%5D=&f%5Bcategory_parent_ids%5D='
    max_page = 733


In [ ]:
# testing response
response = requests.get(page_url)
response.status_code

In [ ]:
# getting all kost url in location
kos_urls = []
html = BeautifulSoup(response.content, 'html.parser')
page = 1
bar = pyprind.ProgBar(max_page, track_time=True, title='Scraping kos urls for {}'.format(loc))
while page <= max_page:
    kos_tags = html.find_all('div', class_='main-column clearfix')
    for kos_url in kos_tags:
        kos_urls.append(kos_url.find('a').get('href'))
    
    next_page_url = html.find('a', class_='button', title='Next page')
    response = requests.get(next_page_url.get('href'))
    html = BeautifulSoup(response.content, 'html.parser')
    page +=1
    bar.update()
    

# saving founded url
kos_url_file = open('url_{}.txt'.format(loc),  'w',  encoding="utf-8")
for url in kos_urls:
    kos_url_file.write('{}\n'.format(url))

## Get kos data


In [2]:
# move list from file of url to list
kos_list = open('url_all.txt',  'r',  encoding="utf-8")
kos_urls = []
for kos in kos_list:
    kos_urls.append(kos)
    
len(kos_urls)

17575

In [13]:
# batch, coz it takes some time
batch1 = kos_urls[:5000]
batch2 = kos_urls[5000:10000]
batch3 = kos_urls[10000:]

print(len(batch1), len(batch2), len(batch3))

5000 5000 7575


In [ ]:
# get data and save it to list variable
# time.sleep with random delay so it wont be recognized as ddos 
titles, owners, jeniss, alamats, no_hps, no_was = [], [], [], [], [], []

data = 'batch1'
if data == 'batch1':
    urls = batch1
elif data == 'batch2':
    urls = batch2
elif data == 'batch3':
    urls = batch3

bar = pyprind.ProgBar(len(urls), track_time=True, title='Scraping {} kos data'.format(data))

for kos_url in urls:
    response = requests.get(kos_url)
    kos_page = BeautifulSoup(response.content, 'html.parser')

    try:
        title = kos_page.find('ol').text
        print(title)
    except:
        title = '-'

    print(title)
    try:
        jenis = kos_page.find(id='df_field_jenis')
        jenis = jenis.find(class_='value').text
        jenis = str(jenis).replace('\n', '')
    except:
        jenis = '-'

    try:
        alamat = kos_page.find(id='df_field_address')
        alamat = alamat.find(class_='value').text
        alamat = str(alamat).replace('\n', '')
    except:
        alamat = '-'

    try:
        owner = kos_page.find(class_='seller-info')
        owner = owner.find(class_='name').text
    except:
        owner = '-'

    try:
        no_hp = kos_page.find(id='df_field_nomorhp')
        no_hp = no_hp.find(class_='value').text
        no_hp = str(no_hp).replace('\n', '')
    except:
        no_hp = '-'

    try:
        no_wa = kos_page.find(id='df_field_whatsapp')
        no_wa = no_wa.find(class_='value').text
        no_wa = str(no_wa).replace('\n', '')
    except:
        no_wa = '-'
    
    titles.append(title)
    jeniss.append(jenis)
    alamats.append(alamat)
    owners.append(owner)
    no_hps.append(no_hp)
    no_was.append(no_wa)    
    delays = [7, 4, 6, 2, 10, 19]
    delay = np.random.choice(delays)
    time.sleep(delay)
    bar.update()

In [8]:
# saving data to file 

kos_title_file = open('.\\scraping\\{}\\title_kos.txt'.format(data),  'w',  encoding="utf-8")
for title in titles:
    kos_title_file.write('{}\n'.format(title))

kos_jenis_file = open('.\\scraping\\{}\\jenis_kos.txt'.format(data),  'w',  encoding="utf-8")
for jenis in jeniss:
    kos_jenis_file.write('{}\n'.format(jenis))

kos_alamat_file = open('.\\scraping\\{}\\alamat_kos.txt'.format(data),  'w',  encoding="utf-8")
for alamat in alamats:
    kos_alamat_file.write('{}\n'.format(alamat))

kos_nohp_file = open('.\\scraping\\{}\\nohp_kos.txt'.format(data),  'w',  encoding="utf-8")
for no_hp in no_hps:
    kos_nohp_file.write('{}\n'.format(no_hp))

kos_nowa_file = open('.\\scraping\\{}\\nowa_kos.txt'.format(data),  'w',  encoding="utf-8")
for no_wa in no_was:
    kos_nowa_file.write('{}\n'.format(no_wa))

kos_owner_file = open('.\\scraping\\{}\\owner_kos.txt'.format(data),  'w',  encoding="utf-8")
for owner in owners:
    owner = owner.replace('\n', '')
    owner = owner.strip()
    if owner != '':        
        kos_owner_file.write('{}\n'.format(owner))
    else:
        kos_owner_file.write('--\n')


## Generate excel from files

In [8]:
kos_list = open('url_all.txt',  'r',  encoding="utf-8")
kos_urls = []
for kos in kos_list:
    kos_urls.append(kos)
        
titles, owners, jeniss, alamats, no_hps, no_was = [], [], [], [], [], []

In [12]:
def add_data(data):
    kos_list = open('url_all.txt',  'r',  encoding="utf-8")
    kos_urls = []
    for kos in kos_list:
        kos_urls.append(kos)
        
    if data == 'batch1':
        urls = kos_urls[:5000]
    elif data == 'batch2':
        urls = kos_urls[5000:10000]
    elif data == 'batch3':
        urls = kos_urls[10000:]

    kos_title_file = open('.\\scraping\\{}\\title_kos.txt'.format(data),  'r',  encoding="utf-8")
    for title in kos_title_file:
        titles.append(title)

    kos_jenis_file = open('.\\scraping\\{}\\jenis_kos.txt'.format(data),  'r',  encoding="utf-8")
    for jenis in kos_jenis_file:
        jeniss.append(jenis)

    kos_alamat_file = open('.\\scraping\\{}\\alamat_kos.txt'.format(data),  'r',  encoding="utf-8")
    for alamat in kos_alamat_file:
        alamats.append(alamat)

    kos_nohp_file = open('.\\scraping\\{}\\nohp_kos.txt'.format(data),  'r',  encoding="utf-8")
    for no_hp in kos_nohp_file:
        no_hps.append(no_hp)

    kos_nowa_file = open('.\\scraping\\{}\\nowa_kos.txt'.format(data),  'r',  encoding="utf-8")
    for no_wa in kos_nowa_file:
        no_was.append(no_wa)

    kos_owner_file = open('.\\scraping\\{}\\owner_kos.txt'.format(data),  'r',  encoding="utf-8")
    for owner in kos_owner_file:
        owners.append(owner)

In [17]:
add_data('batch1')
add_data('batch2')
add_data('batch3')

print(len(titles), len(jeniss), len(alamats), len(owners), len(no_hps), len(no_was), len(kos_urls))

17575 17575 17575 17575 17575 17575 17575


In [22]:
output_path = 'data sewakos.xlsx'
pd.DataFrame({'Judul':titles, 'Jenis Kos':jeniss, 'Alamat Kos':alamats, 'Owner Kos':owners, 'No Hp':no_hps, 'No WA':no_was, 'URL Kos':kos_urls}).to_excel(output_path, index=False)